In [1]:
import numpy as np
import pandas as pd
import bloscpack as bp

In [2]:
import fastai
from fastai.vision import *
# from fastai.callbacks import *

from optim import Over9000

In [3]:
from torch.utils.data.dataloader import DataLoader
from data import DS_TRN

In [4]:
from model import *
from model_utils import *

---
### data

In [5]:
trn_imgs = bp.unpack_ndarray_from_file('../features/train_images.bloscpack')
# see preprocess notebook, it's shown that the train.csv data sequence is the same and can just load like this.
trn_lbls = pd.read_csv('../input/train.csv').iloc[:, 1:4].values

In [6]:
training_set = DS_TRN(trn_imgs[:160000, np.newaxis], trn_lbls)
validation_set = DS_TRN(trn_imgs[160000:, np.newaxis], trn_lbls)

training_loader = DataLoader(training_set, batch_size=64, shuffle=True, num_workers=4)
validation_loader = DataLoader(validation_set, batch_size=64, shuffle=False, num_workers=4)

dat_bunch = DataBunch(train_dl=training_loader, valid_dl=validation_loader)

---
### model

In [7]:
device = 'cuda:0'
n_grapheme = 168
n_vowel = 11
n_consonant = 7
n_total = n_grapheme + n_vowel + n_consonant

In [8]:
predictor = PretrainedCNN(out_dim=n_total)
classifier = BengaliClassifier(predictor)#.to(device)

In [9]:
learn = Learner(
    dat_bunch,
    classifier,
    loss_func=Loss_combine(),
    opt_func=Over9000,
    metrics=[Metric_grapheme(), Metric_vowel(), Metric_consonant(),Metric_tot()]
)

logger = CSVLogger(learn, 'test')

learn.clip_grad = 1.0
learn.split([classifier.predictor.lin_layers])
learn.unfreeze()

In [12]:
learn.fit_one_cycle(
    32,
    max_lr=slice(0.2e-2,1e-2),
    wd=[1e-3, 0.1e-1],
    pct_start=0.0,
    div_factor=100,
    callbacks=[logger, SaveModelCallback(learn,monitor='metric_tot',mode='max', name='test'), MixUpCallback(learn)]
)

epoch,train_loss,valid_loss,metric_idx,metric_idx,metric_idx,metric_tot,time


NameError: name 'learn' is not defined